# Treinamento do Modelo  
Agora que fizemos o tratamento dos dados e a análise, vamos começar a treinar o modelo.  
Lembrando que, o objetivo do modelo é tentar predizer se um determinado paciente vai precisar de um leito de UTI ou não.

## Preparação do Ambiente  

### Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Método para separação de dados
from sklearn.model_selection import StratifiedShuffleSplit

# Encoder para converter valores categóricos
from sklearn.preprocessing import OrdinalEncoder

# Pipeline para encapsulamento do treinamento do modelo
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Modelos que serão testados
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


# Seleção de features
from sklearn.feature_selection import SelectFromModel

# Método para determinar os melhores hiper parâmetros dos modelos
from sklearn.model_selection import GridSearchCV

# Métricas de avaliação do modelo
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate

# Fixando o random seed para reprodutividade
np.random.seed(789872)

### Importando os Dados

In [3]:
dados_clinicos = pd.read_csv('./dados/dados_ajustados.csv')
dados_clinicos.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0,0,0,0,1,1,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
1,2,0,10th,0,0,0,0,0,0,0,...,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,0-2,1
2,3,0,40th,1,0,0,0,0,0,0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
3,4,0,10th,0,0,0,0,0,0,0,...,-0.952381,-0.979798,-1.000000,-0.883669,-0.956805,-0.870968,-0.953536,-0.980333,0-2,0
4,5,0,10th,0,0,0,0,0,0,0,...,-0.976190,-0.979798,-0.860870,-0.714460,-0.986481,-1.000000,-0.975891,-0.980129,0-2,0


In [4]:
#Verificando os dados categóricos no dataset.
dados_clinicos.select_dtypes(include='object')

,AGE_PERCENTIL,WINDOW
0,60th,0-2
1,10th,0-2
2,40th,0-2
3,10th,0-2
4,10th,0-2
...,...,...
348,40th,0-2
349,Above 90th,0-2
350,50th,0-2
351,40th,0-2


Antes, as transformações que são necessárias no dataset:
* Converter a coluna `AGE_PERCENTIL` para valor numérico;
* Retirar as colunas `PATIENT_VISIT_IDENTIFIER` e `WINDOW`, já que eles não são relevantes para predição;

Agora vamos discutir sobre a métrica para avaliação do modelo.

### Métricas para Avaliação  
Para o problema que estamos modelando, não podemos simplesmente utilizar a métrica da precisão. Isso por que, nesse caso é importante também levar em conta que o peso do erro - casos onde o paciente não precisaria de um leito, mas o modelo previu que sim (erro tipo I) e o contrário, ele preciaria do leito e o modelo disse que não (erro tipo II). Se tratando de detecção de doença, esses dois tipos de erros precisam ser levados em conta.  
<br/>
Dito isso, uma das métricas mais utilizadas para problemas de classificação é o [AUC](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc) (Area Under the Curve - Área sob a Curva em português) é uma métrica derivada da ROC (receiver operating characteristic curve) que nos ajuda a entender o quão bom é o nosso modelo para diferenciar as duas classes (precisará de UTI ou não) e é dado pela razão entre a taxa de verdadeiros positivo e falso positivo (TPR / FPR) e o seu valor varia entre 0 e 1. A interpretação do valor é simples - é a probabilidade de, dado duas previsões, elas estarem corretamente classificadas.  
<br/>

Utilizaremos o AUC como métrica principal, mas avaliaremos também o [recall](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score), que mede o quão bem o nosso modelo consegue identificar as classes positivas. Para o nosso case, essa métrica vai nos mostrar o quão bem o nosso modelo consegue evitar os casos de falso negativos (o paciente precisar do leito de UTI e o modelo dizer o contrário).


##### Para saber mais:
* [Matriz de Confusão e AUC ROC](https://medium.com/data-hackers/matriz-de-confus%C3%A3o-e-auc-roc-f7e446dca107);
* [Entenda o que é AUC e ROC nos modelos de Machine Learning](https://medium.com/bio-data-blog/entenda-o-que-%C3%A9-auc-e-roc-nos-modelos-de-machine-learning-8191fb4df772).

## Pré Processamento  
Uma vez definida a métrica, vamos partir para o pre processamento dos dados. Antes de treinar o modelo, precisamos tomar alguns cuidados, como retirar as colunas que não serão usadas como o id do paciênte e a janela (dado que no nosso conjunto o valor é constante), bem como converter os valores categóricos para númericos.

In [5]:
#Realizar a cópia do dataset original para manter a integridade, e dropar as colunas já mencionadas.
pre_processado = dados_clinicos.copy()
pre_processado.drop(['PATIENT_VISIT_IDENTIFIER', 'WINDOW'], axis=1, inplace=True)

### Determinando Método de Separação dos Dados
Antes de selecionarmos o método para separação dos dados de teste e o tamanho, vamos avaliar a distribuição da nossa variável alvo e tamanho do dataset.

In [6]:
pre_processado.shape

(353, 229)

In [7]:
pre_processado['ICU'].value_counts(normalize=True)

0    0.538244
1    0.461756
Name: ICU, dtype: float64

A distribuição dos dados parece estar ligeiramente desbalanceada, o que quer dizer que temos que tomar cuidado para que ao selecionar o conjunto de teste e treinamento a proporção das classes seja mais próxima possível do conjunto total.  
O scikit learn possui vários métodos para separação de dados, e dentre os recomendados para casos de conjunto desbalanceados vamos utilizar o [StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html#sklearn.model_selection.StratifiedShuffleSplit) que, além de respeitar a proporção das classes, ele embaralha os dados antes de separar os dados. Quanto ao tamanho do conjunto de treino e teste, vamos deixar o default do algorítmo.

### Tratamento da coluna AGE_PERCENTIL  
Vamos verificar como os valores estão distribuído no dataset.

In [8]:
pre_processado['AGE_PERCENTIL'].value_counts().sort_index()

10th          38
20th          42
30th          39
40th          38
50th          34
60th          30
70th          34
80th          36
90th          28
Above 90th    34
Name: AGE_PERCENTIL, dtype: int64

Percebam que, essa coluna representa a faixa etária do paciente e essa informação possui uma sequencia lógica, ou seja, tratase de uma `variável quantitativa ordinal`. Como ele possui uma sequencia, podemos utilizar o `OrdinalEncoder` para converter os essa coluna para numérico - ele atribui um valor respeitando a sequencia lógica. Dessa forma o modelo vai entender a importancia de um valor.  
Um adendo: Como essa coluna possui apenas 10 categorias, poderíamos utilizar também o `OneHotEncoder`, que costuma funcionar bem.  
Agora que definimos como trataremos os casos, ao invés de tratar um a um, vamos utilizar a classe Pipeline, que permite encapsular todas as etapas do desenvolvimento do modelo.


## Desenvolvendo o Modelo  
Para avaliarmos o desempenho do modelo e a progressão conforme vamos realizando testes, criaremos um dataframe com os resultados, bem como encapsular numa função todo o processo de desenvolvimento.
Outro detalhe encapsulamos o próprio treinamento do modelo através da classe `Pipeline`, onde passamos as etapas para o desenvolvimento, como o preprocessamento da coluna `AGE_PERCENTIL` e o treinamento do modelo, e através método `cross_validate` realizamos a validação cruzada, embaralhando e testando 10 vezes o conjunto de dados.  
No final, a função devolve a média e o intervalo de confiança do AUC e recall.  
Selecionamos três modelo diferentes para testarmos: regressão logística, SVC e Random Forest:

In [9]:
performance = pd.DataFrame()

In [26]:
def roda_modelo(descricao_modelo : str, dados : pd.DataFrame(), modelo, paramns : dict):
    
    """
    Executa o prcesso de desenvolvimento do modelo:
        1. Separa os features e a variável alvo;
        2. Cria o pre processador a ser acoplado no Pipeline;
        3. Instancia a Pipeline com o pre processador e o modelo passado no argumento model;
        4. Separa os dados através do StratifiedShuffleSplit;
        5. Aplica a validação cruzada e retorna um dataframe com as métricas.
        
    Input:
        * descricao_modelo: Descrição que será aplicado ao modelo no dataframe com o desempenho;
        * dados: dataframe para treinamento e avaliação do modelo;
        * modelo: o algorítimo que irá rodar na pipeline (não instanciar);
        * paramns: dicionário com os hiperparâmetros do modelo (opcional)
        
    return: o dataframe performance preenchido
    
    """
    
    
    X = dados.drop('ICU', axis=1)
    y = dados['ICU']
    
    ordinal_encoder = OrdinalEncoder()

    preprocessor = ColumnTransformer(transformers=[
        ('ordinal', ordinal_encoder, ['AGE_PERCENTIL'])
    ])

    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', modelo(**paramns))
    ])
    
   
    cv = StratifiedShuffleSplit(n_splits=10, random_state=False)
    results = cross_validate(model, X, y, cv=cv, return_train_score=True, scoring=['roc_auc', 'recall'])
    
    auc_medio = results['test_roc_auc'].mean()
    auc_std = results['test_roc_auc'].std()
    recall_medio = results['test_recall'].mean()
    recall_std = results['test_recall'].std()
    
    return performance.append(
        {'model': descricao_modelo,
         'auc_medio' : auc_medio,
         'intervalo' : f'{auc_medio-(2* auc_std) : .3f} - {auc_medio+(2* auc_std) : .3f}',
         'recall_medio' : recall_medio,
         'intervalo recall' : f'\
                    {recall_medio-(2* recall_std) : .3f} - {recall_medio+(2* recall_std) : .3f}'},                             
                             ignore_index=True)
    
    
    

In [11]:
performance = roda_modelo('Regressão Logística', pre_processado, LogisticRegression, {})
performance = roda_modelo('SVC', pre_processado, SVC, {})
performance = roda_modelo('RandomForest', pre_processado, 
                          RandomForestClassifier, {'random_state': False})

performance

,model,auc_medio,intervalo,recall_medio,intervalo recall
0,Regressão Logística,0.765480,0.628 - 0.903,0.658824,0.408 - 0.910
1,SVC,0.720898,0.546 - 0.896,0.570588,0.396 - 0.745
2,RandomForest,0.730495,0.601 - 0.860,0.523529,0.317 - 0.730


Nesse primeiro teste o modelo de regressão logística se saiu melhor que as outras duas. Chama a atenção que o intervalo de confiança ainda está muito largo.  
O próximo passo é tentar melhorar o desempenho do modelo através da feature selection. Os modelos de regressão logística e SVC, que são relativamente simples, tendem a melhorar quando trabalhamos com menos features. Dentre as técnicas disponíveis para realizar esse processo, vamos utilizar o [`SelectFromModel`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html) do scikitlearn, em que consiste criar um meta modelo que selecionará somente as features relevantes.

In [12]:
# Preparando os dados para passar no seletor de features
# Excluindo o AGE_PERCENTIL, que ainda não foi transoformado e iria atrapalhar o modelo.
X = pre_processado.drop(['AGE_PERCENTIL', 'ICU'], axis=1)
y = pre_processado['ICU']

# Modelagem
selector = SelectFromModel(estimator=LogisticRegression()).fit(X, y)

# Retorno de quantas features foram selecionadas.
# a função get_support retorna um array booleana com 
# as colunas que foram selecionadas.
selector.get_support().sum()

85

O modelo que utilizado na seleção foi a regressão logística, por ser um algorítimo relativamente simples e não demandar tanto tempo para processamento.  
Notem que o seletor separou 85 features relevantes, vamos verificar quais são elas.

In [13]:
# Criando máscara com o array selecionado
mask = selector.get_support()

# Aplicando a máscara, e dropando as colunas AGE e ICU, pois no seletor essas colunas
# não existiam - as duas colunas serão inseridas no código seguinte
data_selected = pre_processado.drop(['AGE_PERCENTIL', 'ICU'], axis=1).loc[:, mask]

data_selected['AGE_PERCENTIL'] = pre_processado['AGE_PERCENTIL']
data_selected['ICU'] = pre_processado['ICU']

In [14]:
data_selected.columns

Index(['AGE_ABOVE65', 'GENDER', 'DISEASE GROUPING 1', 'DISEASE GROUPING 2',
       'DISEASE GROUPING 4', 'HTN', 'ALBUMIN_MEAN', 'ALBUMIN_MIN',
       'ALBUMIN_MAX', 'BE_VENOUS_MEDIAN', 'BE_VENOUS_MEAN', 'BE_VENOUS_MIN',
       'BE_VENOUS_MAX', 'BIC_ARTERIAL_MEDIAN', 'BIC_ARTERIAL_MEAN',
       'BIC_ARTERIAL_MIN', 'BIC_ARTERIAL_MAX', 'BIC_VENOUS_MEDIAN',
       'BIC_VENOUS_MEAN', 'BIC_VENOUS_MIN', 'BIC_VENOUS_MAX', 'BLAST_MEDIAN',
       'BLAST_MEAN', 'BLAST_MIN', 'BLAST_MAX', 'CALCIUM_MEDIAN',
       'CALCIUM_MEAN', 'CALCIUM_MIN', 'CALCIUM_MAX', 'CREATININ_MEDIAN',
       'CREATININ_MEAN', 'CREATININ_MIN', 'CREATININ_MAX', 'GLUCOSE_MEDIAN',
       'GLUCOSE_MEAN', 'GLUCOSE_MIN', 'GLUCOSE_MAX', 'HEMATOCRITE_MEDIAN',
       'HEMATOCRITE_MEAN', 'HEMATOCRITE_MIN', 'HEMATOCRITE_MAX', 'INR_MEDIAN',
       'INR_MEAN', 'INR_MIN', 'INR_MAX', 'LINFOCITOS_MEDIAN',
       'LINFOCITOS_MEAN', 'LINFOCITOS_MIN', 'LINFOCITOS_MAX',
       'PC02_VENOUS_MEDIAN', 'PC02_VENOUS_MEAN', 'PC02_VENOUS_MIN',
     

In [15]:
data_selected.head()

,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 4,HTN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,BE_VENOUS_MEDIAN,...,TEMPERATURE_MAX,BLOODPRESSURE_SISTOLIC_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_SISTOLIC_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,AGE_PERCENTIL,ICU
0,1,0,0,0,0,0,0.605263,0.605263,0.605263,-1.0,...,-0.420290,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,60th,1
1,0,0,0,0,0,0,0.605263,0.605263,0.605263,-1.0,...,0.101449,-0.533742,-0.764706,-1.000000,-0.959596,-0.351328,-0.756272,-1.000000,10th,1
2,0,1,0,0,0,0,-0.263158,-0.263158,-0.263158,-1.0,...,-0.420290,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,40th,0
3,0,0,0,0,0,0,0.605263,0.605263,0.605263,-1.0,...,0.072464,-0.877301,-0.882353,-0.952381,-0.979798,-0.883669,-0.870968,-0.953536,10th,0
4,0,0,0,0,0,0,0.605263,0.605263,0.605263,-1.0,...,-0.333333,-0.754601,-1.000000,-0.976190,-0.979798,-0.714460,-1.000000,-0.975891,10th,0


Agora que realizamos a seleção das features, vamos verificar se os modelos performam melhor:

In [16]:
performance = roda_modelo('Regressão log feature selection', data_selected, LogisticRegression, {})
performance = roda_modelo('SVC feature selection ', data_selected, SVC, {})
performance = roda_modelo('RandomForest feature selection', data_selected, 
                          RandomForestClassifier, {'random_state': False})

performance

,model,auc_medio,intervalo,recall_medio,intervalo recall
0,Regressão Logística,0.765480,0.628 - 0.903,0.658824,0.408 - 0.910
1,SVC,0.720898,0.546 - 0.896,0.570588,0.396 - 0.745
2,RandomForest,0.730495,0.601 - 0.860,0.523529,0.317 - 0.730
3,Regressão log feature selection,0.765480,0.628 - 0.903,0.658824,0.408 - 0.910
4,SVC feature selection,0.720898,0.546 - 0.896,0.570588,0.396 - 0.745
5,RandomForest feature selection,0.730495,0.601 - 0.860,0.523529,0.317 - 0.730


E notasse que o desempenho foi a mesma do teste anterior, nenhum modelo apresentou melhora no desempanho.  
Agora vamos verificar se a tunagem dos hiperparâmtros pode nos ajudar. Para isso vamos utilizar a biblioteca [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Esse método recebe um conjunto de hiperparâmetros e testa todas as combinações para determinar os melhores valores para cada hiperparâmetros. Um problema que pode surgir e que temos que ficar de olho é o tempo de execução para rodar esse código - quando mais hiperparâmetros passarmos para serem testados, mais tempo ele irá demorar para achar a melhor combinação, uma vez que ele irá testar um por um.  
Novamente, vamos criar uma função para automatizar essa etapa.

In [17]:
def find_best_params(model, dados, parameters):
    
    """
    Recebe o modelo, dados e o dicionário de hiperpârametros para serem testados.
    Métrica de avaliação: AUC.
    
    model: Modelo a ser testado;
    dados: Dataset para realizar o teste;
    parameters: dicionário contendo os hiperparâmetros que serão testados.
    
    return: dicionário com o conjundo dos melhores parametros.
    
    """
    
    X = dados.drop('ICU', axis=1)
    y = dados['ICU']
    
    ordinal_encoder = OrdinalEncoder()
    
    preprocessor = ColumnTransformer(transformers=[
        ('ordinal', ordinal_encoder, ['AGE_PERCENTIL'])
    ])

    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model())
    ])
    
    cv = StratifiedShuffleSplit(n_splits=10, random_state=False)
    grid = GridSearchCV(model, param_grid=parameters, cv=cv, scoring='roc_auc', n_jobs=1)    
    grid.fit(X, y)
    
    return grid.best_params_

Agora que definimos a função, vamos rodar a função.  
Note que no dicionário que vamos passar para a função temos que adicionar 'model__' antes do nome do hiperparâmetro. Isso porque com o grid search podemos verificar a tunagem de outras etapas contidas no pipeline, se assim fosse necessário - como dentro do pipe chamamos o modelo de `model`, precisamos especificar que queremos testar os hiperparâmetros do modelo.  
Para podermos passar os hiperparâmetros para o próximo modelo, vamos salvar esse dicionário, aplicando uma transormação antes para retirar a string 'model__' das chaves.

In [18]:
def organiza_parametros(dicionario):
    lista_chave = list(dicionario)
    for chave in lista_chave:
        nova_chave = chave.replace('model__', '')
        dicionario[nova_chave] = dicionario.pop(chave)
        
    return dicionario

In [19]:
parametros_log = {
    'model__penalty' : ('l1', 'l2', 'elasticnet'),
    'model__C' : [0.01, 0.1, 1, 10, 100, 1000],
    
}

melhores_parametros_log = find_best_params(LogisticRegression, data_selected, 
                                              parameters=parametros_log)

melhores_parametros_log = organiza_parametros(melhores_parametros_log)
melhores_parametros_log

{'C': 0.01, 'penalty': 'l2'}

In [20]:
parametros_svc = {
    'model__kernel' : ('linear', 'rbf', 'sigmoid'),
    'model__gamma' : [0.01, 0.1, 1, 10, 100],
    'model__C' : [0.01, 0.1, 1, 10, 100],
}

melhores_parametros_svc = find_best_params(SVC, data_selected, parameters=parametros_svc)
melhores_parametros_svc = organiza_parametros(melhores_parametros_svc)
melhores_parametros_svc

{'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}

In [21]:
parametros_floresta = {
    'model__n_estimators' : [5, 10, 50, 100],
    'model__criterion': ('gini', 'entropy'),
    'model__min_samples_split': [4, 6, 8],
    'model__max_depth' : [2, 3, 5, 6, 8],
}

melhores_parametros_floresta = find_best_params(RandomForestClassifier, data_selected, 
                                                parameters=parametros_floresta)

melhores_parametros_floresta = organiza_parametros(melhores_parametros_floresta)
melhores_parametros_floresta

{'criterion': 'entropy',
 'max_depth': 2,
 'min_samples_split': 6,
 'n_estimators': 100}

Agora que temos os parametros selecionados, vamos rodar novamnte.

In [22]:
performance = roda_modelo('Regressão log - Grid', data_selected, 
                          LogisticRegression, melhores_parametros_log)

performance = roda_modelo('SVC - grid', data_selected, SVC, melhores_parametros_svc)

performance = roda_modelo('RandomForest - grid', data_selected, 
                          RandomForestClassifier, melhores_parametros_floresta)

performance

,model,auc_medio,intervalo,recall_medio,intervalo recall
0,Regressão Logística,0.765480,0.628 - 0.903,0.658824,0.408 - 0.910
1,SVC,0.720898,0.546 - 0.896,0.570588,0.396 - 0.745
2,RandomForest,0.730495,0.601 - 0.860,0.523529,0.317 - 0.730
3,Regressão log feature selection,0.765480,0.628 - 0.903,0.658824,0.408 - 0.910
4,SVC feature selection,0.720898,0.546 - 0.896,0.570588,0.396 - 0.745
5,RandomForest feature selection,0.730495,0.601 - 0.860,0.523529,0.317 - 0.730
6,Regressão log - Grid,0.765480,0.628 - 0.903,0.623529,0.334 - 0.913
7,SVC - grid,0.765480,0.628 - 0.903,0.664706,0.418 - 0.912
8,RandomForest - grid,0.762384,0.610 - 0.914,0.523529,0.317 - 0.730


E aqui notamos que não houve melhoras no modelo de regressão mesmo com a tunagem, e o auc médio de 0.76 parece ser o limite com os dados que temos, já que o SVC alcançou a mesma pontuação, o interessante é que o intervalo de confiança foi alcançou o mesmo valor. Quanto ao modelo de random forest, apresentou uma melhora significativa, mas não conseguiu chegar na mesma pontuação, apesar de ter quase alcançado.  
Sendo assim, a classificação dos modelos testados ficam assim:

In [23]:
performance.sort_values(['auc_medio', 'recall_medio'], ascending=False)

,model,auc_medio,intervalo,recall_medio,intervalo recall
7,SVC - grid,0.765480,0.628 - 0.903,0.664706,0.418 - 0.912
0,Regressão Logística,0.765480,0.628 - 0.903,0.658824,0.408 - 0.910
3,Regressão log feature selection,0.765480,0.628 - 0.903,0.658824,0.408 - 0.910
6,Regressão log - Grid,0.765480,0.628 - 0.903,0.623529,0.334 - 0.913
8,RandomForest - grid,0.762384,0.610 - 0.914,0.523529,0.317 - 0.730
2,RandomForest,0.730495,0.601 - 0.860,0.523529,0.317 - 0.730
5,RandomForest feature selection,0.730495,0.601 - 0.860,0.523529,0.317 - 0.730
1,SVC,0.720898,0.546 - 0.896,0.570588,0.396 - 0.745
4,SVC feature selection,0.720898,0.546 - 0.896,0.570588,0.396 - 0.745


### Mas qual modelo escolher?  
Olhando somente para o AUC, podemos considerar que houve um empate técnico entre os três modelos, mas olhando para o recall vemos que o modelo SVC conseguiu performar um pouco melhor que o modelo de regressão logística.  
Porém, o resultado não entregou um modelo satisfatório. O desempenho de todos foram abaixo da expectativa e o intervalo de confiança está muito grande - com uma variação tão grande, não podemos confiar na decisão dele para auxiliar no planejamento de leitos de UTI.

## Resultado e Discussão  
O modelo desenvolvido ainda não está performando bem suficiente para ser colocado em produção, na realidade com um intervalo de confiança com uma variação tão grande nas duas métricas não podemos afirmar se ele está bem ou não. Uma das hipóteses é que, com um dataset pequeno (353 pacientes) os nossos modelos não foi capaz de generalizar bem e entregar um resultado consistente, portanto seria interessante agregar mais dados para assim podemos avaliar melhor.  
Outro ponto que pode afetar o nosso modelo - e que não foi explorado nesse projeto, é o feature engineering. Grande parte das features do dataset são oriundas dessa técnica, aplicada pela própria equipe de dados do hospital, mas dada o baixo desempenho, investigar mais e tentar criar novas fetures pode trazer melhores resultados.  
E por fim a forma que lidamos com a limpeza dos dados pode ter afetado o resultado. Lembrando: selecionamos somente a primeira janela (0-2) por paciente e, caso os valores fossem nulos, fazia um preenchimento com os dados do próprio paciente, só que das janelas posteriores. Avaliar outras formas de preenchimento desses dados vazios pode ser interessante. 
Olhando para o intervalo de confiança, podemos ver que o modelo ainda possui grande margem para melhoria e por isso vale a pena trabalhar nesse projeto.